In [ ]:
import pandas as pd
data = pd.read_csv('/kaggle/input/medical-text/train.dat',delimiter='\t')

In [ ]:
import spacy
import string
nlp = spacy.load("en_core_web_lg")
stop_words = nlp.Defaults.stop_words
print(stop_words)
punctuations = string.punctuation
print(punctuations)
punctuations = '!\"#$%&\'()*+,-/:;<=>?@[\]^_`{|}~'

In [ ]:
def spacy_tokenizer(sentence):
    doc = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    mytokens = [ word for word in mytokens if word not in punctuations ]
    sentence = " ".join(mytokens)
    return sentence

In [ ]:
import nltk
from nltk import tokenize

def entailment(data):
    data = data.iloc[:500]
    data['cleaned'] = data['text'].apply(spacy_tokenizer)
    data=data.assign(scombi="")

    for c in range(len(data)):
        try:
            sentence_tok = []
            sentence_tok = tokenize.sent_tokenize(data["cleaned"][c])
            if sentence_tok[-1].endswith('.'):
                sentence_tok =  sentence_tok
            else:
                sentence_tok[-1] = str(sentence_tok[-1]) + str('.')
            data['scombi'][c] = '  '.join(sentence_tok)
           
        except:
            print('1')
    data = data.assign(pairs="")    
        
    import itertools
    for c in range(len(data)):
        token = data.scombi[c].split('  ')
        data.pairs[c] = list(itertools.combinations(token,2))

    df_list_comp = pd.DataFrame()
    test_list = [y for c in range(len(data.pairs)) for y in (data.pairs[c])]
    df_list_comp = pd.DataFrame(test_list,columns = ['sentence1','sentence2'])
    df_list_comp = df_list_comp.assign(label = 'entailment')
    for i in range(len(df_list_comp)):
        if len(df_list_comp.sentence1[i].split(' ')) < 5 or len(df_list_comp.sentence2[i].split(' ')) < 5 :
            df_list_comp= df_list_comp.drop(i,axis=0)
    df_list_comp.reset_index(drop=True)
    return df_list_comp

In [ ]:
import random 
def neutral(data):
    data = data.iloc[:500]
    data['cleaned'] = data['text'].apply(spacy_tokenizer)
    data=data.assign(scombi="")

    for c in range(len(data)):
        try:
            sentence_tok = []
            sentence_tok = tokenize.sent_tokenize(data["cleaned"][c])
            if sentence_tok[-1].endswith('.'):
                sentence_tok =  sentence_tok
            else:
                sentence_tok[-1] = str(sentence_tok[-1]) + str('.')
            data['scombi'][c] = '  '.join(sentence_tok)
           
        except:
            print('1')
    data1 = data.iloc[:250] 
    data2 = data.iloc[250:]  
    data2 = data2.reset_index(drop=True)
    list1 = []
    list2 = []
    # df1 = pd.DataFrame()
    # df2 = pd.DataFrame()
    for c in range(len(data1)):
        token1 = data1.scombi[c].split('  ')
        token2 = data2.scombi[c].split('  ')
        list1.extend(token1)
        list2.extend(token2)
        # df1 = pd.DataFrame(list1,columns = ['single_sentences'])
        # df2 = pd.DataFrame(list2,columns = ['single_sentences'])
    x=0
    for i in range(len(list1)):
        if len(list1[i-x].split(' ')) < 5:
            list1.pop(i-x)
            x=x+1
    x=0
    for i in range(len(list2)):
        if len(list2[i-x].split(' ')) < 5:
            list2.pop(i-x)
            x=x+1
    # for i in range(len(df2)):
    #     if len(df2.single_sentences[i].split(' ')) < 5:
    #         df2= df2.drop(i,axis=0)
    neutral_list = [(q1,q2) for q1 in list1 for q2 in random.sample(list2,k=13)]
    df_neutral = pd.DataFrame(neutral_list,columns = ['sentence1','sentence2'])
    df_neutral = df_neutral.assign(label='neutral')
    return df_neutral

In [ ]:
import random 
def contradiction(data,data2):
    data = data.iloc[:50]
    data2 = data2.iloc[:50]
    data['cleaned'] = data['text'].apply(spacy_tokenizer)
    data2['cleaned'] = data2['text'].apply(spacy_tokenizer)
    data=data.assign(scombi="")
    data2=data2.assign(scombi="")

    for c in range(len(data)):
        try:
            sentence_tok = []
            sentence_tok2 = []
            sentence_tok = tokenize.sent_tokenize(data["cleaned"][c])
            sentence_tok2 = tokenize.sent_tokenize(data2["cleaned"][c])
            if sentence_tok[-1].endswith('.'):
                sentence_tok =  sentence_tok
            else:
                sentence_tok[-1] = str(sentence_tok[-1]) + str('.')
            if sentence_tok2[-1].endswith('.'):
                sentence_tok2 =  sentence_tok2
            else:
                sentence_tok2[-1] = str(sentence_tok2[-1]) + str('.')
            data['scombi'][c] = '  '.join(sentence_tok)
            data2['scombi'][c] = '  '.join(sentence_tok2)
           
        except:
            print('1')
    list1 = []
    list2 = []
    # df1 = pd.DataFrame()
    # df2 = pd.DataFrame()
    for c in range(len(data)):
        token1 = data.scombi[c].split('  ')
        token2 = data2.scombi[c].split('  ')
        list1.extend(token1)
        list2.extend(token2)
        # df1 = pd.DataFrame(list1,columns = ['single_sentences'])
        # df2 = pd.DataFrame(list2,columns = ['single_sentences'])
    x=0
    for i in range(len(list1)):
        if len(list1[i-x].split(' ')) < 5:
            list1.pop(i-x)
            x=x+1
    x=0
    for i in range(len(list2)):
        if len(list2[i-x].split(' ')) < 5:
            list2.pop(i-x)
            x=x+1
    # for i in range(len(df2)):
    #     if len(df2.single_sentences[i].split(' ')) < 5:
    #         df2= df2.drop(i,axis=0)
    contradiction_list = [(q1,q2) for q1 in list1 for q2 in random.sample(list2,k=35)]
    df_contradiction = pd.DataFrame(contradiction_list,columns = ['sentence1','sentence2'])
    df_contradiction = df_contradiction.assign(label='contradiction')
    return df_contradiction


In [ ]:
#call for dataframes then concat

In [ ]:
import random
lst = list(range(len(data)))
random.seed(5)
random_list = random.sample(lst,5000)

In [ ]:
sorted(random_list)[:5]

In [ ]:
data= data.assign(split="train")

for z in random_list:
    data.loc[z,'split'] = 'eval'
data.head(20)

In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.evaluation import LabelAccuracyEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import os
import gzip
import csv
import pandas as pd

label2int = {"contradiction": 0, "entailment":1,"neutral":2}
train_samples_my=[]
dev_samples_my=[]
for i in range(len(data)):
    label_id = label2int[data['label'][i]]
    if data['split'][i] == 'train':
        train_samples_my.append(InputExample(texts=[data['sentence1'][i],data['sentence2'][i]],label=label_id))
    else:
        dev_samples_my.append(InputExample(texts=[data['sentence1'][i],data['sentence2'][i]],label=label_id))


In [ ]:
import math
train_batch_size = 16
model_save_path = './model'
from sentence_transformers import models, SentenceTransformer
#bert_bio = models.Transformer('dmis-lab/biobert-v1.1')
#pooler = models.Pooling(bert_bio.get_word_embedding_dimension(),pooling_mode_mean_tokens = True)
#model = SentenceTransformer(modules=[bert_bio, pooler])
model= SentenceTransformer('all-MiniLM-L6-v2')

from torch.utils.data import DataLoader
batch_size=16
loader = DataLoader(train_samples_my,batch_size,shuffle=True)
dev_loader = DataLoader(dev_samples_my,batch_size,shuffle=True)

from sentence_transformers import losses

loss = losses.SoftmaxLoss(model=model,num_labels=3,sentence_embedding_dimension=384)
evaluator_my = LabelAccuracyEvaluator(dev_loader, name="medical_eval")


In [ ]:
epochs = 1
warmup_steps = int(len(loader) * epochs * 0.1)
model.fit(train_objectives=[(loader, loss)],
          
          epochs = epochs,
          warmup_steps = warmup_steps,
      
          output_path='./triple_loss_all_1000'
          )

In [ ]:
data = pd.read_pickle('')
data = data.sample(frac=1).reset_index(drop=True)
data_test = pd.read_csv('')
data_test = data_test.drop_duplicates()
data_test = data_test.reset_index(drop =True)


classes=data['Category'].unique().tolist()
nb_classes = len(classes)
print(nb_classes)
print(classes)


for i,type_c in enumerate(classes):
   for j,type_t in enumerate(data['Category']):
       if type_c == type_t :
           data.loc[j,'Category'] = i
for i,type_c in enumerate(classes):
   for j,type_t in enumerate(data_test['Category']):
       if type_c == type_t :
           data_test.loc[j,'Category'] = i
           
           
data['embeddings'] = data['text'].apply(model.encode)
data_test['embeddings'] = data_test['text'].apply(model.encode)

In [ ]:
from sklearn.model_selection import train_test_split
X_train = data['embeddings'].to_list()
y_train = data['Category'].to_list()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y)
X_test = data_test['embeddings'].to_list()
y_test = data_test['Category'].to_list()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

clfs = [
    ('LogisticRegression', LogisticRegression(max_iter=3000,
                                              class_weight='balanced')
    ),
    ('RandomForest', RandomForestClassifier(max_depth=18,
                                            n_estimators=75,
                                            random_state=0)
    ),
    ('KNN 5', KNeighborsClassifier(n_neighbors=4)
    ),
    ('SVM C1', SVC(C=1,
                   class_weight='balanced')
    )]

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score


def print_val_scores(scores: list[float]) -> None:

  print(f'Cross validation scores: mean: {np.mean(scores):.3f}, '
        f'all: {[round(score, 3) for score in scores]}')


def print_stratified_kfold(clfs: list[tuple[str, any]], X_train: pd.DataFrame,
                           y_train: pd.Series, n_splits: int = 5, cv: int = 5,
                           ) -> None:

  for clf in clfs:
    print(f'\nStratifiedKFold - classifier: {clf[0]}:\n')
    skf = StratifiedKFold(n_splits=n_splits)

    scores = cross_val_score(clf[1],
                            X_train,
                            y_train,
                            cv=cv)

    print_val_scores(scores)

In [ ]:
print_stratified_kfold(clfs, X_train, y_train)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, classification_report

clf =  SVC(C=1,class_weight='balanced')

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
# prob = clf.predict_proba(X_test)
# print(prob)

accuracy = np.mean(y_pred == y_test)

ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.title(f'SVM - acc {accuracy:.3f}', size=15)
plt.show()